


# TP protection des données : Implémentation de RSA avec Gmpy2




### Installation de Conda

In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


### Afficher la version de conda

In [ ]:
!conda –version


usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: '–version' (choose from 'activate', 'deactivate', 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'doctor', 'repoquery', 'env')


### Regarder où ce situe le dossier conda

In [ ]:
!which conda

/usr/local/bin/conda


### Installation de Gmpy2 avec Conda

In [ ]:
!conda install -c conda-forge gmpy2

Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



### Importation de packages

In [ ]:
from gmpy2 import is_prime , next_prime , mpz_urandomb, is_strong_prp,random_state,mpz_random,invert,powmod
import time
from math import gcd

### Génération de nombres premiers

In [ ]:
def get_prime(size): # size la taille en binaire
    while True :
       r = random_state(time.time_ns())
       p = mpz_urandomb(r,size)
       p = p.bit_set(size-1)
       return next_prime(p)









In [ ]:
# Test :
x = get_prime(512)
print("x = ", x, x.bit_length())
print("is x prime ?", is_prime(x))
y = get_prime(512)
print("y = ", y, y.bit_length())
print("is y prime ?", is_prime(y))
z = x*y
print("la longuer de z en bits =", z.bit_length())

x =  10858239649078636428497323923083502195295886259487495065889812706747463198655079829309811706931935958198110765150662023225569386269987593613906824505994167 512
is x prime ? True
y =  10847808329916712539154703340230811521570678098647561833470927392414198789562611092052116714191780095902294492235800637028619974204097518659691502111378921 512
is y prime ? True
la longuer de z en bits = 1024


### Génération de clé publique et privée de RSA

In [ ]:
# Implémenter une fonction qui prend en paramètre la taille de la clé publique et qui retourne la clé publique et privée de RSA
def get_rsa_keys(size):
    p = get_prime(int(size/2))
    while True:
      q = get_prime(int(size/2))
      if q != p :
        break
    n = p*q
    phi = (p-1)*(q-1)
    while True:
      seed = random_state (time.time_ns())
      e = mpz_random(seed,phi)
      if gcd (e,phi) == 1:
        break
    d = invert (e,phi)
    pub_key = [n,e]
    priv_key = [d , max (p,q), min (p,q)]
    return pub_key, priv_key


### Fonctions de chiffrement et de déchiffrement

In [ ]:
# Implémenter la fonction de chiffrement RSA et la fonction de déchiffrement avec et sans CRT.
def rsa_encrypt(message, pub_key):

    c = powmod (message,pub_key[1],pub_key[0])
    return c

def rsa_decrypt(enc, priv_key, pub_key):
    message = powmod (enc,priv_key[0],pub_key[0])
    return message

def rsa_decrypt_crt(enc, priv_key, pub_key):
    p = priv_key [1]
    q = priv_key [2]
    d = priv_key [0]
    dp = powmod (d,1,p-1)
    dq= powmod (d,1,q-1)
    qinv = invert (q,p)
    m1 = powmod (enc,dp,p)
    m2 = powmod (enc,dq,q)
    h = powmod (qinv*(m1-m2),1,p)
    m = m1 + h* q
    return m

### Test RSA

In [ ]:
pub_key, priv_key = get_rsa_keys(1024)
print("e==", pub_key[0])
print("N==", pub_key[1].bit_length())
print("d==", priv_key[0])

e== 75556900234944491452185755430929332727017090041852347415011522283486445774955551054596140023339813062242598055163974465379491599661227510783624705054425189096005810989865765940202360589631608260184980043629392266734908764982521819373689979235231106019878895438182176653662949249423648212695393771285257787181
N== 1021
d== 70506254090413218526698626924232963521234595223715431239976263011642196159696239034664130348140651999220469131437132823613729944302471445905650311257093175217165022476220367734052398535816895799482174677330340279566103019059121826416936399602577978449226146654572348116521848131475040279893802969806332919555


In [ ]:
message = mpz(102132100)
print("clear message = ", message)
enc = rsa_encrypt(message, pub_key)
print("encrypted message = ", enc)
m = rsa_decrypt(enc, priv_key, pub_key)
print("decreypted message = ", m)
m = rsa_decrypt_crt(enc, priv_key, pub_key)
print("CRT decrypted message = ", m)

clear message =  102132100
encrypted message =  10051833906862578982906696131366155629071159515704784213023270720718172292907979357583242428804180405801489554206364070284610141722355071323060489892933267986906161385926502245324671736412145970249505955851075444484612788659088965307537514726773277318015849765978496809287109368624064891036663439358304292700
decreypted message =  102132100
CRT decrypted message =  102132100
